# Individual Project: Unsupervised CT Denoising with CycleGAN
This notebook is provided for unsupervised CT denoising task with cycleGAN.

###Table of Contents

I. Data: Google Drive, Dataloader

II. Network: Generator / Discriminator

III. Other functions

IV. Training

V. Test

## I. Data

### 1. Google Drive
If you want to use data in your Google drive, you have to mount your google drive first.
It makes possible to load data from the drive, save results in the drive, etc. 
If you try to mount Google Drive, a link for authentication code will be given. 
Then, follow the link, get your authentication code, and enter your code in the prompt.

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load an example image
img = np.load('/content/drive/My Drive/AAPM_data/train/full_dose/1.npy')

# Change linear attenuation coefficient into HU values
img = (img - 0.0192) / 0.0192 * 1000
# Clip the CT image with [-1000, 1000] HU
img = np.clip(img, -1000, 1000)

# Plot the example image
plt.imshow(img, cmap='gray')
plt.show()

### 2. Dataloader
Before the training of cycleGAN, data preprocessing (e.g. cliping, normalization) is required.


In [ ]:
import random
from os import listdir
from os.path import join
from torch.utils.data import Dataset


# CT dataset
class CT_Dataset(Dataset):
  def __init__(self, path, transform):
    # Path of 'full_dose' and 'quarter_dose' folders
    self.path_full = join(path, 'full_dose')
    self.path_quarter = join(path, 'quarter_dose')
    self.transform = transform

    # File list of full dose data
    self.file_full = list()
    for file_name in sorted(listdir(self.path_full)):
      self.file_full.append(file_name)
    random.seed(0)
    random.shuffle(self.file_full)
    
    # File list of quarter dose data
    self.file_quarter = list()
    for file_name in sorted(listdir(self.path_quarter)):
      self.file_quarter.append(file_name)
  
  def __len__(self):
    return min(len(self.file_full), len(self.file_quarter))
  
  def __getitem__(self, idx):
    # Load full dose/quarter dose data
    x_F = np.load(join(self.path_full, self.file_full[idx]))
    x_Q = np.load(join(self.path_quarter, self.file_quarter[idx]))

    # Convert to HU scale
    x_F = (x_F - 0.0192) / 0.0192 * 1000
    x_Q = (x_Q - 0.0192) / 0.0192 * 1000

    # Normalize images
    x_F[x_F < -1000] = -1000
    x_Q[x_Q < -1000] = -1000

    x_F = x_F / 4000
    x_Q = x_Q / 4000

    # Apply transform
    x_F = self.transform(x_F)
    x_Q = self.transform(x_Q)

    file_name = self.file_quarter[idx]

    return x_F, x_Q, file_name

Because the size of CT images is too large, we have to crop the images into small size patches for training.
After training with patches, the whole size image will be used for the test.

In [ ]:
import random
import torchvision
from torch.utils.data import DataLoader


# Transform for the random crop
class RandomCrop(object):
  def __init__(self, patch_size):
    self.patch_size = patch_size
  
  def __call__(self, img):
    # Randomly crop the image into a patch with the size [self.patch_size, self.patch_size]
    w, h = img.size(-1), img.size(-2)
    i = random.randint(0, h - self.patch_size)
    j = random.randint(0, w - self.patch_size)

    return img[:, i:i + self.patch_size, j:j + self.patch_size]


# Make dataloader for training/test
def make_dataloader(path, batch_size):
  # Path of 'train' and 'test' folders
  path_train = join(path, 'train')
  path_test = join(path, 'test')

  # Transform for training data: convert to tensor, random horizontal/verical flip, random crop
  # You can change transform if you want.
  train_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    RandomCrop(128)
  ])

  # Transform for test data: convert to tensor
  test_transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
  ])

  # Generate CT dataset for training/test
  train_dataset = CT_Dataset(path_train, train_transform)
  test_dataset = CT_Dataset(path_test, test_transform)
  
  # Generate dataloader for training/test
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
  test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True)

  return train_dataloader, test_dataloader

## II. Network: Generator / Discriminator
You have to implement basic blocks for generator/discriminator.
Refer to the 'ConvBlock', and make your own blocks for building generators/discriminators.

In [ ]:
import torch
import torch.nn as nn


# Example convolution block. You don't have to use this block.
# (Convolution, Batch normalization, ReLU) x 2
class ConvBlock(nn.Module):
  def __init__(self, in_channels, mid_channels, out_channels):
    super(ConvBlock, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, mid_channels, kernel_size=3, stride=1, padding=1, bias=False),
        nn.BatchNorm2d(mid_channels, affine=True, track_running_stats=True),
        nn.ReLU(),
        nn.Conv2d(mid_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
        nn.BatchNorm2d(out_channels, affine=True, track_running_stats=True),
        nn.ReLU()
    )
  
  def forward(self, x):
    out = self.conv(x)
    return out
  
# You can implement other building blocks for making generator and discriminator.
######################
# Your code
######################

### 1. Generator
Now, implement the generator and discriminator below.
You are free to choose the structure of the generator (e.g. U-Net, ResNet), but it should contain residual path.

In [ ]:
# Generator
class Generator(nn.Module):
  def __init__(self, in_channels, out_channels, ngf):
    super(Generator, self).__init__()
    # in_channels: the number of channels of the input
    # out_channels: the number of channels of the output
    # ngf: the number of convolution filters of the first layer
    ######################
    # Your code
    ######################
  
  def forward(self, x):
    ######################
    # Your code
    ######################

    # Residual path: final output = output + input
    ######################
    # Your code
    ######################

### 2. Discriminator
You have to construct PatchGAN structure for the discriminator as shown in PPT slide.

In [ ]:
# Discriminator (PatchGAN)
class Discriminator(nn.Module):
  def __init__(self, in_channels, ndf):
    super(Discriminator, self).__init__()
    # in_channels: the number of channels of the input
    # ndf: the number of convolution filters of the first layer
    ######################
    # Your code
    ######################
  
  def forward(self, x):
    ######################
    # Your code
    ######################

## III. Other functions


In [ ]:
from torch.nn import init


# initialize parameters of neural networks
def init_weights(net):
  def init_func(m):
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
      init.normal_(m.weight.data, 0.0, 0.02)
      if hasattr(m, 'bias') and m.bias is not None:
        init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm2d') != -1:
      init.normal_(m.weight.data, 1.0, 0.02)
      init.constant_(m.bias.data, 0.0)
    
  print('Initialize network.')
  net.apply(init_func)


# Calculate average loss during one epoch
class Mean:
  def __init__(self):
    self.numel = 0
    self.mean = 0
  
  def __call__(self, val):
    self.mean = self.mean * (self.numel / (self.numel + 1)) + val / (self.numel + 1)
    self.numel += 1
  
  def result(self):
    return self.mean


# Show input and output images during training
def show_imgs(imgs):
  FQF = np.concatenate(imgs[:3], axis=2)
  QFQ = np.concatenate(imgs[3:], axis=2)
  img_array = np.squeeze(np.concatenate([FQF, QFQ], axis=1))

  img_array = img_array * 4000
  img_array = np.clip(img_array, -1000, 1000)

  plt.imshow(img_array, cmap='gray')
  plt.show()


# Set 'requires_grad' of the networks
def set_requires_grad(nets, requires_grad=False):
  if not isinstance(nets, list):
    nets = [nets]
  for net in nets:
    if net is not None:
      for param in net.parameters():
        param.requires_grad = requires_grad

## IV. Training
Before training the network, some hyperparameters should be defined as follows.
You can change the value of hyperparameters if you want.

In [ ]:
from os import makedirs
from os.path import isdir

# Hyperparameters
# You can change hyperparameters to find your best performance in your architecture.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 16
lambda_cycle = 10
lambda_iden = 5
beta1 = 0.5
beta2 = 0.999
num_epoch = 100
lr = 2e-4

# Path for saving the checkpoint
path_checkpoint = '/content/drive/My Drive/CT_denoising'
if not isdir(path_checkpoint):
  makedirs(path_checkpoint)

```model_name``` is the name of the model, and it will be used for saving the model.
If you want to continue the training from the last checkpoint, set ```model_name``` as the name of the saved model.
However, if you want to train a new model, you have to change ```model_name```.

In [ ]:
model_name = 'cyclegan_v1'

# Path for saving results
path_result = join(path_checkpoint, model_name)
if not isdir(path_result):
  makedirs(path_result)

Next, make dataloaders, networks, optimizers, and define loss functions.

In [ ]:
import itertools

# path of dataset (change the path according to your setting)
path_data = '/content/drive/My Drive/AAPM_data'

# Make dataloaders
train_dataloader, test_dataloader = make_dataloader(path_data, batch_size)

# Make generators (G_F2Q: full to quarter / G_Q2F: quarter to full)
####################
G_F2Q = # your code
G_Q2F = # your code
####################

# Make discriminators (D_F: distinguish real/fake full dose images / D_Q: distinguish real/fake quarter dose images)
####################
D_F = # your code
D_Q = # your code
####################

# Make optimizers
G_optim = torch.optim.Adam(itertools.chain(G_F2Q.parameters(), G_Q2F.parameters()), lr, betas=(beta1, beta2))
D_optim = torch.optim.Adam(itertools.chain(D_F.parameters(), D_Q.parameters()), lr, betas=(beta1, beta2))

# Define loss functions
adv_loss = nn.MSELoss()
cycle_loss = nn.L1Loss()
iden_loss = nn.L1Loss()

# Loss functions
loss_name = ['G_adv_loss_F',
             'G_adv_loss_Q',
             'G_cycle_loss_F',
             'G_cycle_loss_Q',
             'G_iden_loss_F',
             'G_iden_loss_Q',
             'D_adv_loss_F',
             'D_adv_loss_Q']

In [ ]:
from os.path import isfile
from tqdm.auto import tqdm


# Load the last checkpoint if it exists
if isfile(join(path_checkpoint, model_name + '.pth')):
  checkpoint = torch.load(join(path_checkpoint, model_name + '.pth'))
  G_F2Q.load_state_dict(checkpoint['G_F2Q_state_dict'])
  G_Q2F.load_state_dict(checkpoint['G_Q2F_state_dict'])
  D_F.load_state_dict(checkpoint['D_F_state_dict'])
  D_Q.load_state_dict(checkpoint['D_Q_state_dict'])
  G_optim.load_state_dict(checkpoint['G_optim_state_dict'])
  D_optim.load_state_dict(checkpoint['D_optim_state_dict'])
  trained_epoch = checkpoint['epoch']
  losses_list = {name: torch.load(join(path_result, name + '.npy')) for name in loss_name}
  print('Start from save model - ' + str(trained_epoch))
# If the checkpoint does not exist, start the training with random initialized model
else:
  init_weights(G_F2Q)
  init_weights(G_Q2F)
  init_weights(D_F)
  init_weights(D_Q)
  trained_epoch = 0
  losses_list = {name: list() for name in loss_name}
  print('Start from random initialized model')

for epoch in tqdm(range(trained_epoch, num_epoch), desc='Epoch', total=num_epoch, initial=trained_epoch):
  losses = {name: Mean() for name in loss_name}

  for x_F, x_Q, _ in tqdm(train_dataloader, desc='Step'):
    x_F = x_F.to(device)
    x_Q = x_Q.to(device)

    # Set 'requires_grad' of the discriminators as 'False'
    ####################
    # Your code
    ####################

    x_FQ = # your code
    x_QF = # your code
    x_QFQ = # your code
    x_FQF = # your code
    x_QQ = # your code
    x_FF = # your code

    G_adv_loss_F = # your code
    G_adv_loss_Q = # your code
    G_cycle_loss_F = # your code
    G_cycle_loss_Q = # your code
    G_iden_loss_F = # your code
    G_iden_loss_Q = # your code
    G_adv_loss = G_adv_loss_F + G_adv_loss_Q
    G_cycle_loss = G_cycle_loss_F + G_cycle_loss_Q
    G_iden_loss = G_iden_loss_F + G_iden_loss_Q
    G_total_loss = G_adv_loss_F + G_adv_loss_Q + lambda_cycle * (G_cycle_loss) + lambda_iden * (G_iden_loss)

    G_optim.zero_grad()
    G_total_loss.backward()
    G_optim.step()
    
    # Set 'requires_grad' of the discriminators as 'True'
    ####################
    # Your code
    ####################

    # You have to detach the outputs of the generators in below codes
    D_adv_loss_F = # your code
    D_adv_loss_Q = # your code
    D_total_loss_F = D_adv_loss_F / 2.0
    D_total_loss_Q = D_adv_loss_Q / 2.0

    D_optim.zero_grad()
    D_total_loss_F.backward()
    D_total_loss_Q.backward()
    D_optim.step()

    # Calculate the average loss during one epoch
    losses['G_adv_loss_F'](G_adv_loss_F.detach())
    losses['G_adv_loss_Q'](G_adv_loss_Q.detach())
    losses['G_cycle_loss_F'](G_cycle_loss_F.detach())
    losses['G_cycle_loss_Q'](G_cycle_loss_Q.detach())
    losses['G_iden_loss_F'](G_iden_loss_F.detach())
    losses['G_iden_loss_Q'](G_iden_loss_Q.detach())
    losses['D_adv_loss_F'](D_adv_loss_F.detach())
    losses['D_adv_loss_Q'](D_adv_loss_Q.detach())
  
  for name in loss_name:
    losses_list[name].append(losses[name].result())
  
  # Save the trained model and list of losses
  torch.save({'epoch': epoch + 1, 'G_F2Q_state_dict': G_F2Q.state_dict(), 'G_Q2F_state_dict': G_Q2F.state_dict(),
              'D_F_state_dict': D_F.state_dict(), 'D_Q_state_dict': D_Q.state_dict(),
              'G_optim_state_dict': G_optim.state_dict(), 'D_optim_state_dict': D_optim.state_dict()}, join(path_checkpoint, model_name + '.pth'))
  for name in loss_name:
    torch.save(losses_list[name], join(path_result, name + '.npy'))
  
  # Plot input/output images every 10 epochs
  if (epoch + 1) % 10 == 0:
    imgs = [x_F[0].detach().cpu().numpy(), x_FQ[0].detach().cpu().numpy(), x_FQF[0].detach().cpu().numpy(),
            x_Q[0].detach().cpu().numpy(), x_QF[0].detach().cpu().numpy(), x_QFQ[0].detach().cpu().numpy()]
    show_imgs(imgs)

## V. Test
Last, you have to verify the performance of your network.

In [ ]:
from tqdm.auto import tqdm

# Load the last checkpoint
checkpoint = torch.load(join(path_checkpoint, model_name + '.pth'))
G_Q2F.load_state_dict(checkpoint['G_Q2F_state_dict'])
G_Q2F.eval()

# Test and save
with torch.no_grad():
  for _, x_Q, file_name in tqdm(test_dataloader):
    x_Q = x_Q.to(device)
    x_QF = G_Q2F(x_Q)[0].detach().cpu().numpy()
    x_QF = x_QF * 4000

    np.save(join(path_result, file_name[0]), x_QF)

In [ ]:
import matplotlib.pyplot as plt

# Plot loss graph (adversarial loss)
x_axis = np.arange(1, num_epoch + 1)
plt.figure(1)
for name in ['G_adv_loss_F', 'G_adv_loss_Q', 'D_adv_loss_F', 'D_adv_loss_Q']:
  loss_arr = torch.load(join(path_result, name + '.npy'))
  plt.plot(x_axis, loss_arr, label=name)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc='upper right')
plt.savefig(join(path_result, 'loss_curve_1.png'))
plt.show()

# Plot loss graph (cycle consistency loss, identity loss)
plt.figure(2)
for name in ['G_cycle_loss_F', 'G_cycle_loss_Q', 'G_identity_loss_F', 'G_identity_loss_Q']:
  loss_arr = torch.load(join(path_result, name + '.npy'))
  plt.plot(x_axis, loss_arr, label=name)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc='upper right')
plt.savefig(join(path_result, 'loss_curve_2.png'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Example of result
path_quarter = join(path_data, 'test/quarter_dose/100.npy')
path_full = join(path_data, 'test/full_dose/100.npy')
path_output = join(path_result, '100.npy')

quarter = np.load(path_quarter)
full = np.load(path_full)
output = np.load(path_output)

quarter = (quarter - 0.0192) / 0.0192 * 1000
full = (full - 0.0192) / 0.0192 * 1000

quarter = np.clip(quarter, -1000, 1000)
full = np.clip(full, -1000, 1000)
output = np.clip(output, -1000, 1000)

plt.imshow(quarter, cmap='gray')
plt.show()
plt.imshow(full, cmap='gray')
plt.show()
plt.imshow(output, cmap='gray')
plt.show()

In [ ]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Functions for caculating PSNR, SSIM
def psnr(A, ref):
  ref[ref < -1000] = -1000
  A[A < -1000] = -1000
  val_min = -1000
  val_max = np.amax(ref)
  ref = (ref - val_min) / (val_max - val_min)
  A = (A - val_min) / (val_max - val_min)
  out = peak_signal_noise_ratio(ref, A)
  return out

def ssim(A, ref):
  ref[ref < -1000] = -1000
  A[A < -1000] = -1000
  val_min = -1000
  val_max = np.amax(ref)
  ref = (ref - val_min) / (val_max - val_min)
  A = (A - val_min) / (val_max - val_min)
  out = structural_similarity(ref, A, data_range=2)
  return out

In [ ]:
# How to use functions 'psnr' and 'ssim'
path_quarter = join(path_data, 'test/quarter_dose/100.npy')
path_full = join(path_data, 'test/full_dose/100.npy')
path_output = join(path_result, '100.npy')

quarter = np.load(path_quarter)
full = np.load(path_full)
output = np.load(path_output)

quarter = (quarter - 0.0192) / 0.0192 * 1000
full = (full - 0.0192) / 0.0192 * 1000

print(psnr(quarter, full))
print(ssim(quarter, full))
print(psnr(output, full))
print(ssim(output, full))